In [9]:
import os
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint

In [5]:
batch_size = 32
# Data directory
train_directory = 'DB/train'
test_directory = 'DB/test'

# Loading train set + Applying data augmentation on train set
train_datagen = ImageDataGenerator(rescale=1./255,
        rotation_range=20,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.2,
        brightness_range=[0.8,1.0],
        fill_mode="nearest")

# Loading test set
test_datagen = ImageDataGenerator(rescale=1./255)

In [3]:
# Generating train batches
train_batches = train_datagen.flow_from_directory(
        train_directory,
        target_size=(224,224),
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle=True)

# Generating test batches
test_batches = test_datagen.flow_from_directory(
        test_directory,
        target_size=(224,224),
        batch_size=batch_size,
        color_mode="rgb",
        class_mode='categorical',
        shuffle=True)

Found 4736 images belonging to 5 classes.
Found 3568 images belonging to 5 classes.


In [4]:
# Model architecture 
# We used pretrained model MobileNetV2 and loaded ImageNet's weight
mobilenet = MobileNetV2(weights = 'imagenet', include_top = True, input_shape=(224,224,3))
mobilenet.layers.pop()
model = Sequential()
model.add(mobilenet)
model.add(Dense(5, activation='softmax', name='predictions'))
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1000)              3538984   
_________________________________________________________________
predictions (Dense)          (None, 5)                 5005      
Total params: 3,543,989
Trainable params: 3,509,877
Non-trainable params: 34,112
_________________________________________________________________


In [6]:
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [7]:
# Model training
num_train = 4736
num_val = 3568
num_epoch = 10

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['acc'])
model.fit(
        train_batches,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=test_batches,
        validation_steps=num_val // batch_size,
        callbacks=[model_checkpoint_callback])

Epoch 1/10
148/148 [==============================] - 480s 3s/step - loss: 1.5670 - acc: 0.7035 - val_loss: 1.5734 - val_acc: 0.4617
Epoch 2/10
148/148 [==============================] - 444s 3s/step - loss: 1.5157 - acc: 0.9113 - val_loss: 1.5551 - val_acc: 0.5180
Epoch 3/10
148/148 [==============================] - 453s 3s/step - loss: 1.4877 - acc: 0.9367 - val_loss: 1.5254 - val_acc: 0.6520
Epoch 4/10
148/148 [==============================] - 437s 3s/step - loss: 1.4645 - acc: 0.9493 - val_loss: 1.5240 - val_acc: 0.6081
Epoch 5/10
148/148 [==============================] - 437s 3s/step - loss: 1.4430 - acc: 0.9578 - val_loss: 1.4717 - val_acc: 0.7950
Epoch 6/10
148/148 [==============================] - 418s 3s/step - loss: 1.4209 - acc: 0.9662 - val_loss: 1.4505 - val_acc: 0.8322
Epoch 7/10
148/148 [==============================] - 411s 3s/step - loss: 1.4000 - acc: 0.9738 - val_loss: 1.4527 - val_acc: 0.7680
Epoch 8/10
148/148 [==============================] - 414s 3s/step - 

In [24]:
# Model architecture 
# We used pretrained model MobileNetV2 and loaded ImageNet's weight
modelbis = Sequential()
modelbis.add(mobilenet)
modelbis.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1000)              3538984   
Total params: 3,538,984
Trainable params: 0
Non-trainable params: 3,538,984
_________________________________________________________________


In [25]:
for layer in modelbis.layers : 
    layer.trainable = False 


In [26]:
modelbis.add(Dense(5, activation='softmax', name='predictions'))

In [27]:
modelbis.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_224 (Functi (None, 1000)              3538984   
_________________________________________________________________
predictions (Dense)          (None, 5)                 5005      
Total params: 3,543,989
Trainable params: 5,005
Non-trainable params: 3,538,984
_________________________________________________________________


In [28]:
modelbis.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['acc'])
modelbis.fit(
        train_batches,
        steps_per_epoch=num_train // batch_size,
        epochs=num_epoch,
        validation_data=test_batches,
        validation_steps=num_val // batch_size)

Epoch 1/10
148/148 [==============================] - 88s 592ms/step - loss: 1.6234 - acc: 0.1005 - val_loss: 1.6187 - val_acc: 0.1689
Epoch 2/10
148/148 [==============================] - 94s 637ms/step - loss: 1.6075 - acc: 0.2196 - val_loss: 1.6062 - val_acc: 0.3063
Epoch 3/10
148/148 [==============================] - 100s 673ms/step - loss: 1.5931 - acc: 0.4223 - val_loss: 1.5942 - val_acc: 0.3345
Epoch 4/10
148/148 [==============================] - 101s 685ms/step - loss: 1.5789 - acc: 0.5338 - val_loss: 1.5814 - val_acc: 0.4381
Epoch 5/10
148/148 [==============================] - 96s 652ms/step - loss: 1.5632 - acc: 0.5693 - val_loss: 1.5679 - val_acc: 0.4437
Epoch 6/10
148/148 [==============================] - 89s 604ms/step - loss: 1.5482 - acc: 0.7052 - val_loss: 1.5527 - val_acc: 0.6104
Epoch 7/10
148/148 [==============================] - 84s 569ms/step - loss: 1.5335 - acc: 0.7179 - val_loss: 1.5423 - val_acc: 0.5957
Epoch 8/10
148/148 [==============================] -